<a href="https://colab.research.google.com/github/Pmilivojevic/PyTorch/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.nn.modules import padding

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class ResBlock(nn.Module):
  def __init__(
      self,
      in_channels=64,
      out_channels=64,
      identity_downsampling=None,
      stride=1
  ):
    super(ResBlock, self).__init__()

    self.expansion = 4
    self.identity_downsampling = identity_downsampling

    self.conv1 = nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=1,
        stride=1,
        padding=0
    )
    self.conv2 = nn.Conv2d(
        out_channels,
        out_channels,
        kernel_size=3,
        stride=stride,
        padding=1
    )
    self.conv3 = nn.Conv2d(
        out_channels,
        out_channels*self.expansion,
        kernel_size=1,
        stride=1,
        padding=0
    )

    self.bn1 = nn.BatchNorm2d(out_channels)
    self.bn2 = nn.BatchNorm2d(out_channels*self.expansion)

    self.relu = nn.ReLU()

  def forward(self, x):
    identity = x.clone()

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.conv2(x)
    x = self.bn1(x)
    x = self.conv3(x)
    x = self.bn2(x)

    if self.identity_downsampling is not None:
      identity = self.identity_downsampling(identity)

    x += identity
    x = self.relu(x)

    return x

In [ ]:
class ResNet(nn.Module):
  def __init__(self, block, layers, image_channels=3, num_classes=1000):
    super(ResNet, self).__init__()

    self.in_channels = 64

    self.conv1 = nn.Conv2d(
        image_channels,
        64,
        kernel_size=7,
        stride=2,
        padding=3
    )
    self.bn = nn.BatchNorm2d(64)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    self.res_layer1 = self._make_layer(
        block,
        layers[0],
        out_channels=64,
        stride=1
    )
    self.res_layer2 = self._make_layer(
        block,
        layers[1],
        out_channels=128,
        stride=2,
    )
    self.res_layer3 = self._make_layer(
        block,
        layers[2],
        out_channels=256,
        stride=2
    )
    self.res_layer4 = self._make_layer(
        block,
        layers[3],
        out_channels=512,
        stride=2
    )
    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512*4, num_classes)

  def _make_layer(self, block, num_res_blocks, out_channels, stride):
    identity_downsampling = None
    layers = []

    if stride != 1 or self.in_channels != out_channels*4:
      identity_downsampling = nn.Sequential(
          nn.Conv2d(
              self.in_channels,
              out_channels*4,
              kernel_size=1,
              stride=stride
          ),
          nn.BatchNorm2d(out_channels*4)
      )

      layers.append(block(
          self.in_channels,
          out_channels,
          identity_downsampling,
          stride
      ))

      self.in_channels = out_channels*4

    for i in range(num_res_blocks-1):
      layers.append(block(self.in_channels, out_channels))

    return nn.Sequential(*layers)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.res_layer1(x)
    x = self.res_layer2(x)
    x = self.res_layer3(x)
    x = self.res_layer4(x)

    x = self.avgpool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc(x)

    return x

In [ ]:
resnet50_conf = [3, 4, 6, 3]
resnet101_conf = [3, 4, 23, 3]
resnet152_conf = [3, 4, 36, 3]

model = ResNet(ResBlock, resnet152_conf)
x = torch.randn(2, 3, 224, 224)
print(model(x).shape)

torch.Size([2, 1000])
